# Construcción del dataframe base

##### A partir los archivos JSON guardados en la base de datos, crear un dataframe modelo como base para el análisis de datos, creación de graficas y herramientas.

In [1]:
# Importamos las librerias necesarias

import pandas as pd
import certifi
import pymongo as py
import os
from dotenv import load_dotenv # Get env variables
load_dotenv()

mongo_uri = os.getenv('MONGO_URI')

In [2]:
# Creamos una variable con la conexion a la base de datos. 

from pymongo import MongoClient
cartonadoradores = MongoClient(mongo_uri)

# Seleccionamos la BD previamente creada en mongo shell y con las validaciones ya configuradas.

df = cartonadoradores['cartonadoradores']

In [3]:
# Importamos todas las coleeciones de las que obtenemos información.

usuarios = df['Usuarios']
productos = df['Productos']
ordenes = df['ordenesCompra']

In [4]:
# Todas las colecciones son recibidas como archivos JSON

for x in usuarios.find():
    print (x)

{'_id': ObjectId('5fba998a345a4bb6ed3446c8'), 'apellidoContacto': 'Gonzalez', 'nombreContacto': 'Enrique', 'email': 'onoYoko@dominio.com', 'calle': 'Calle ', 'numero': 'No.6 Local 1', 'alcaldia_municipio': 'Cuahutemoc', 'ciudad': 'CDMX', 'cp': 6400, 'rfc': 'LOQ64O4O1GHO', 'nivelAcceso': 'USER', 'razonSocial': 'Cliente loqay', 'password': 'e10adc3949ba59abbe56e057f20f883e'}
{'_id': ObjectId('5fbb2a25169e8c0534716100'), 'apellidoContacto': 'Rodriguez  ', 'nombreContacto': 'Rosario', 'email': 'facturaciones@quesito.com.mx', 'calle': 'Calle', 'numero': 'No. 43', 'alcaldia_municipio': 'Texcoco', 'ciudad': 'Estado de México', 'cp': 56170, 'rfc': 'CCO81O31166O', 'nivelAcceso': 'USER', 'razonSocial': 'QUESITOS S.A. DE C.V.', 'password': 'e10adc3949ba59abbe56e057f20f883e'}
{'_id': ObjectId('5fbb463ad94852d0d6906924'), 'apellidoContacto': 'Parangueo', 'nombreContacto': 'Norma', 'email': 'invernaderos@izzi.com.mx', 'calle': 'Calle', 'numero': 'No. 16', 'alcaldia_municipio': 'Zitacuaro', 'ciudad':

In [5]:
    # En una variable asignamos el codigo pipeline que indica las agregaciones necesarias para crear nuestro dataset
    # a través de una vista

pipeline = [
    {
        '$unwind': {
            'path': '$Productos'
        }
    }, {
        '$lookup': {
            'from': 'Productos', 
            'localField': 'Productos.idProducto', 
            'foreignField': '_id', 
            'as': 'Especificaciones'
        }
    }, {
        '$unwind': {
            'path': '$Especificaciones'
        }
    }, {
        '$lookup': {
            'from': 'Usuarios', 
            'localField': 'idUsuario', 
            'foreignField': '_id', 
            'as': 'Usuario'
        }
    }, {
        '$unwind': {
            'path': '$Usuario'
        }
    }, {
        '$addFields': {
            'priceKey': {
                '$concat': [
                    {
                        '$toString': '$Especificaciones.calibreECT'
                    }, '$Especificaciones.tipo', '$Especificaciones.caraExterior'
                ]
            }
        }
    }]

Por medio del metodo _.aggregate()_, ejecutamos las agregaciones con la variable el pipeline como parametro.

Usamos el metodo _json_normalize()_ para convertir la lista en un dataframe con un registro por cada producto vendido en cada orden.

In [6]:
dfJSON = list(df.ordenesCompra.aggregate(pipeline))
operaciones = pd.json_normalize(dfJSON)

Usamos el metodo _.groupby()_ para mostrar los clientes existentes y usar la información en los filtros siguientes.

In [7]:
operaciones.groupby('Especificaciones.estructura').last()

,_id,idUsuario,tipo,estado,foliointerno,fechaEmision,fechaCierre,priceKey,Productos.idProducto,Productos.idLocal,...,Usuario.email,Usuario.calle,Usuario.numero,Usuario.alcaldia_municipio,Usuario.ciudad,Usuario.cp,Usuario.rfc,Usuario.nivelAcceso,Usuario.razonSocial,Usuario.password
Especificaciones.estructura,,,,,,,,,,,,,,,,,,,,,
caja en dos piezas,5fcbdc7e4124c8597eb6c8e3,5fba998a345a4bb6ed3446c8,ordenProduccion,abierta,23383,2020-02-27,2020-04-20,32dosPiezaskraft,5fc3dd7e30024fb085af64ad,CJ247,...,onoYoko@dominio.com,Calle,No.6 Local 1,Cuahutemoc,CDMX,6400,LOQ64O4O1GHO,USER,Cliente loqay,e10adc3949ba59abbe56e057f20f883e
caja regular,5fcbf3144124c8597eb6c910,5fba998a345a4bb6ed3446c8,ordenProduccion,abierta,23202,2020-01-28,2020-03-24,32regularkraft,5fc3d5a130024fb085af64ab,CJ246,...,onoYoko@dominio.com,Calle,No.6 Local 1,Cuahutemoc,CDMX,6400,LOQ64O4O1GHO,USER,Cliente loqay,e10adc3949ba59abbe56e057f20f883e
charola,5fc980408e8ca1720f06b590,5fbb56b3cfeb74a0ee851d57,ordenProduccion,abierta,0000001731,2020-10-23,2020-10-26,32N/Akraft,5fc9425462e33f09f0bd8a30,CAJACHAROLAOW1D,...,mobiliario@modulares.com.mx,Calle,No. 1102 Piso 2 202,Miguel hidalgo,CDMX,11200,IMA191187008,USER,Cliente Intermuebles,e10adc3949ba59abbe56e057f20f883e
separador,5fc94c7ceab151fb539d0ede,5fbb2a25169e8c0534716100,ordenProduccion,abierta,3685-354,2017-10-27,2017-11-15,26N/Akraft,5fc3f6b030024fb085af64b6,panela premium separador,...,facturaciones@quesito.com.mx,Calle,No. 43,Texcoco,Estado de México,56170,CCO81O31166O,USER,QUESITOS S.A. DE C.V.,e10adc3949ba59abbe56e057f20f883e
suaje,5fcbc8fa4124c8597eb6c8cf,5fbb56b3cfeb74a0ee851d57,ordenProduccion,abierta,0000000167,2020-11-29,2020-12-02,32regularkraft,5fcbc5446966fe2679e7feab,CAJA35BWBE,...,mobiliario@modulares.com.mx,Calle,No. 1102 Piso 2 202,Miguel hidalgo,CDMX,11200,IMA191187008,USER,Cliente Intermuebles,e10adc3949ba59abbe56e057f20f883e


In [8]:
operaciones.columns

Index(['_id', 'idUsuario', 'tipo', 'estado', 'foliointerno', 'fechaEmision',
       'fechaCierre', 'priceKey', 'Productos.idProducto', 'Productos.idLocal',
       'Productos.unidades', 'Productos.precioUnitario', 'Productos.cantidad',
       'Productos.importe', 'Especificaciones._id', 'Especificaciones.idLocal',
       'Especificaciones.estructura', 'Especificaciones.descripcion',
       'Especificaciones.tipo', 'Especificaciones.calibreECT',
       'Especificaciones.caraExterior', 'Especificaciones.largo',
       'Especificaciones.ancho', 'Especificaciones.alto',
       'Especificaciones.tipoUnion', 'Especificaciones.suaje', 'Usuario._id',
       'Usuario.apellidoContacto', 'Usuario.nombreContacto', 'Usuario.email',
       'Usuario.calle', 'Usuario.numero', 'Usuario.alcaldia_municipio',
       'Usuario.ciudad', 'Usuario.cp', 'Usuario.rfc', 'Usuario.nivelAcceso',
       'Usuario.razonSocial', 'Usuario.password'],
      dtype='object')

Creamos un dataframe filtrado por compañia para doble checar los detalles de las ordenes de compra por cada cliente y verificar que no haya repetidos

In [9]:
prueba = operaciones['Usuario.razonSocial'] == 'LOQUAY. S.A.'

#operaciones[prueba].groupby('foliointerno')[['_id','Especificaciones.calibreECT','Especificaciones.largo',
 #     'Especificaciones.ancho', 'Especificaciones.alto','Productos.precioUnitario', 'Especificaciones.descripcion', 'Productos.idProducto']].last()
    
operaciones.groupby(['Usuario.razonSocial']).agg({'_id': 'count', 'Productos.importe':'sum', 'Productos.cantidad': 'sum', 'Productos.idLocal': 'count'})    
    

,_id,Productos.importe,Productos.cantidad,Productos.idLocal
Usuario.razonSocial,,,,
Cliente Intermuebles,29,991275.0,20.725,29
Cliente Zitacuaro,12,867600.0,48.000,12
Cliente loqay,79,1455258.0,226.000,79
Cliente plasticos.,9,270935.0,25.500,9
QUESITOS S.A. DE C.V.,106,6467325.0,530.000,106


Usamos el metodo _groupby()_ sobre el dataframe de prueba para verificar que los productos asociados a cada cliente no se repitan y esten correctamente ordenados con las caracteristicas correctas.

In [10]:
operaciones[prueba].groupby('Productos.idProducto')[['_id', 'Productos.idLocal', 'Especificaciones.calibreECT','Especificaciones.largo', \
'Especificaciones.ancho', 'Especificaciones.alto','Productos.precioUnitario', 'Especificaciones.descripcion', 'Productos.idProducto']].last()


,_id,Productos.idLocal,Especificaciones.calibreECT,Especificaciones.largo,Especificaciones.ancho,Especificaciones.alto,Productos.precioUnitario,Especificaciones.descripcion,Productos.idProducto
Productos.idProducto,,,,,,,,,


Una vez limpio el dataset lo exportamos con el metodo _.to_csv_ para ubicarlo y utilizarlo facilmente

In [11]:
operaciones.to_csv('../analisis/operaciones.csv')